The term hate speech is understood as any type of verbal, written or behavioural communication that attacks or uses derogatory or discriminatory language against a person or group based on what they are, in other words, based on their religion, ethnicity, nationality, race, colour, ancestry, sex or another identity factor. In this article, I will take you through a hate speech detection model with Machine Learning and Python.

Hate Speech Detection is generally a task of sentiment classification. So for training, a model that can classify hate speech from a certain piece of text can be achieved by training it on a data that is generally used to classify sentiments. So for the task of hate speech detection model, I will use the Twitter data.


## Hate Speech Detection Model
The data set I will use for the hate speech detection model consists of a test and train set. The training package includes a list of 31,962 tweets, a corresponding ID and a tag 0 or 1 for each tweet. The particular sentiment we need to detect in this dataset is whether or not the tweet is based on hate speech.

So, let’s get started with the task of building a hate speech detection model. I will simply start with reading the datasets by using the pandas package in python:

In [12]:
import numpy as np
import pandas as pd
import re

In [13]:
train = pd.read_csv(r'C:\Users\amany\Desktop\archive datasets\hatespeech\train.csv')
print("Train Data :\n",train.columns,'\n',train.shape,)

test = pd.read_csv(r'C:\Users\amany\Desktop\archive datasets\hatespeech\test.csv')
print("\n\nTest Data :\n",test.columns,'\n',test.shape,)

Train Data :
 Index(['id', 'label', 'tweet'], dtype='object') 
 (31962, 3)


Test Data :
 Index(['id', 'tweet'], dtype='object') 
 (17197, 2)


In [14]:
print(train.isnull().sum(),'\n') 
print(test.isnull().sum())

id       0
label    0
tweet    0
dtype: int64 

id       0
tweet    0
dtype: int64


In [15]:
train.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [16]:
test.head()

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,@user #white #supremacists want everyone to s...
2,31965,safe ways to heal your #acne!! #altwaystohe...
3,31966,is the hp and the cursed child book up for res...
4,31967,"3rd #bihday to my amazing, hilarious #nephew..."


In [17]:
train['label'].value_counts()

0    29720
1     2242
Name: label, dtype: int64

## Data Cleaning
Data cleaning is the process of preparing incorrectly formated data for analysis by deleting or modifying the incorrectly formatted data which is generally not necessary or useful for data analysis, as it can hinder the process or provide inaccurate results. Now I will perform the process of data cleaning by using the re library in Python:

In [19]:
def cleantext(df,tweet):
    df[tweet]=df[tweet].str.lower()
    df[tweet]=df[tweet].apply(lambda ele:re.sub('[^A-Za-z]',' ',ele))
    return df
test_clean = cleantext(test, "tweet")
train_clean = cleantext(train, "tweet")

In [21]:
train_clean

,id,label,tweet
0,1,0,user when a father is dysfunctional and is s...
1,2,0,user user thanks for lyft credit i can t us...
2,3,0,bihday your majesty
3,4,0,model i love u take with u all the time in ...
4,5,0,factsguide society now motivation
...,...,...,...
31957,31958,0,ate user isz that youuu ...
31958,31959,0,to see nina turner on the airwaves trying to...
31959,31960,0,listening to sad songs on a monday morning otw...
31960,31961,1,user sikh temple vandalised in in calgary ...


## Handling Imbalanced data for Hate Speech Detection Model
If you will deeply analyse the task we are working on with context to the data we are using, you will find that the tweets regarding hate speeches are comparatively lesser than others, so this is a situation of an unbalanced data.

If we will fit this data to train our hate speech detection model, then the model will not generalize any hate speech because the data with context to the hate speech is very less than the positive ones. So in this situation, we need to prepare the data to fit properly in our model.

There are a number of methods you can use to deal with this. One approach is to use either oversampling or downsampling. In the case of oversampling, we use a function that repeatedly samples, with replacement, from the minority class until the class is the same size as the majority. Let’s see how we can handle this:

In [23]:
from sklearn.utils import resample
train_majority = train_clean[train_clean.label==0]
train_minority = train_clean[train_clean.label==1]
train_minority_unsampled=resample(train_minority,
                                 replace=True,
                                 n_samples=len(train_majority),
                                 random_state=123)
train_unsampled=pd.concat([train_minority_unsampled,train_majority])
train_unsampled['label'].value_counts()

0    29720
1    29720
Name: label, dtype: int64

## Creating a Pipeline
For simplicity and reproducibility of the hate speech detection model, I will use the Scikit-Learn’s pipeline with an SGDClassifier, before training our model:

In [29]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer,TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier

pipeline_sgd=Pipeline([
    ('vect',CountVectorizer()),
    ('tfidf',TfidfTransformer()),
    ('nb',SGDClassifier())])

## Training the Hate Speech Detection Model
Now, before training the model, let’s split the data into a training set and a test set

In [30]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(train_unsampled['tweet'],train_unsampled['label'],random_state=0)

In [31]:
X_train.shape,X_test.shape

((44580,), (14860,))

In [32]:
model = pipeline_sgd.fit(X_train,y_train)
y_predict = model.predict(X_test)

In [33]:
from sklearn.metrics import f1_score
f1_score(y_test,y_predict)

0.9655585903670642

So we got an F1 score of 0.96 per cent which is generally appreciatable. Use F1-Score when deal with Imbalanced Dataset